In [1]:
import os
import requests
import pandas as pd
import time
from tqdm import tqdm
from datetime import datetime, timedelta
import ssl
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
load_dotenv()

True

# 연결 테스트

In [ ]:
# 1단계: 필요한 라이브러리 임포트 및 SSL 설정
import requests
import urllib.parse
import ssl
import warnings
import os
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
load_dotenv()

# 경고 메시지 억제
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# SSL 컨텍스트 전역 변경 (핵심 해결책)
ssl._create_default_https_context = ssl._create_unverified_context

print("✅ SSL 설정 완료")

# 2단계: API 키 및 기본 설정
API_KEY = os.getenv('DO_API_KEY')
BASE_URL = 'http://apis.data.go.kr/B552845/katSale/trades'

print("✅ API 설정 완료")

# 3단계: 간단한 연결 테스트
def test_connection():
    """연결 테스트"""
    print("\n🔍 연결 테스트 중...")
    
    params = {
        'serviceKey': API_KEY,
        'pageNo': 1,
        'numOfRows': 1
    }
    
    try:
        response = requests.get(BASE_URL, params=params, verify=False, timeout=15)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ 연결 성공!")
            try:
                data = response.json()
                print(f"응답 데이터 타입: {type(data)}")
                if isinstance(data, dict):
                    print(f"응답 키: {list(data.keys())}")
                return True
            except:
                print("⚠️ JSON 파싱 실패, 하지만 연결은 성공")
                print(f"응답 내용: {response.text[:200]}...")
                return True
        else:
            print(f"❌ 연결 실패: {response.status_code}")
            print(f"응답: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return False

# 4단계: 실제 데이터 조회 함수
def get_market_data(date='2024-01-15', market_code='110001', large_cat='06', mid_cat='01'):
    """시장 데이터 조회"""
    print(f"\n📊 데이터 조회 시작...")
    print(f"   날짜: {date}")
    print(f"   시장코드: {market_code}")
    print(f"   대분류: {large_cat}, 중분류: {mid_cat}")
    
    params = {
        'serviceKey': API_KEY,
        'pageNo': 1,
        'numOfRows': 10,
        'cond[trd_clcln_ymd::EQ]': date,
        'cond[whsl_mrkt_cd::EQ]': market_code,
        'cond[gds_lclsf_cd::EQ]': large_cat,
        'cond[gds_mclsf_cd::EQ]': mid_cat
    }
    
    try:
        response = requests.get(BASE_URL, params=params, verify=False, timeout=15)
        print(f"Status Code: {response.status_code}")
        print(f"URL: {response.url}")
        
        if response.status_code == 200:
            try:
                json_data = response.json()
                print("✅ 데이터 조회 성공!")
                
                # 응답 구조 분석
                print(f"\n📋 응답 구조:")
                if isinstance(json_data, dict):
                    for key, value in json_data.items():
                        if isinstance(value, list):
                            print(f"  {key}: 리스트 (길이: {len(value)})")
                        else:
                            print(f"  {key}: {type(value).__name__}")
                
                # 데이터 출력
                print(f"\n📄 전체 응답:")
                from pprint import pprint
                pprint(json_data)
                
                return json_data
                
            except json.JSONDecodeError as e:
                print(f"❌ JSON 파싱 오류: {e}")
                print(f"응답 내용: {response.text}")
                return None
        else:
            print(f"❌ API 호출 실패: {response.status_code}")
            print(f"응답 내용: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 요청 중 오류 발생: {e}")
        return None

# 5단계: 실행
print("🚀 한국농수산식품유통공사 API 테스트 시작")
print("=" * 60)

# 연결 테스트 실행
if test_connection():
    print("\n" + "="*60)
    # 실제 데이터 조회 실행
    result = get_market_data()
    
    if result:
        print("\n🎉 모든 테스트 성공!")
    else:
        print("\n🔄 다른 날짜로 재시도...")
        # 다른 날짜로 시도
        for test_date in ['2023-12-15', '2023-11-15', '2023-10-15']:
            print(f"\n📅 {test_date}로 재시도...")
            result = get_market_data(date=test_date)
            if result:
                break
else:
    print("\n❌ 연결 테스트 실패")
    print("🔧 추가 해결 방안:")
    print("1. 방화벽 또는 회사 네트워크 정책 확인")
    print("2. VPN 연결 해제 후 재시도")
    print("3. 다른 네트워크 환경에서 테스트")
    print("4. API 키 재발급 확인")

# 추출코드

In [ ]:
# SSL 및 경고 설정
warnings.filterwarnings('ignore', category=InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

# 디렉토리 준비

os.makedirs("data/fail_log", exist_ok=True)

# API 설정
API_KEY = os.getenv('DO_API_KEY')  # 환경 변수에서 키를 불러옵니다
BASE_URL = 'http://apis.data.go.kr/B552845/katSale/trades'

if not API_KEY:
    raise ValueError("환경변수 'DO_API_KEY'가 설정되지 않았습니다.")

# 도매시장 코드 불러오기
df_market = pd.read_csv('도매시장_코드.csv', encoding='cp949')

# 품목 코드 설정
ITEM_CODES = {
#    "양파": "1201",  # 진성
#     "배추": "1001",  # 용곤
#     "상추": "1005",  # 동현
#     "사과": "0601"   # 재성
    "무" : '1101',
    "감자" : '0501',
#     "대파" : "1202",
#     "건고추" : "1207",
#     "마늘" : "1209",
#     "딸기" : '0804',
#     "방울토마토" : "0806",
#     "오이" : "0901",
#     "양배추" : "1004",
#     "고구마" : '0502',
#     '배' : '0602'
}

# 휴일 일자 불러오기
df_date = pd.read_csv("holiday_score_table.csv", encoding="cp949")
df_holiday = df_date[df_date['holiday_flag']==1]
holidays = df_holiday['date'].values

# 날짜 입력 (YYYY-MM-DD)  # 테스트 후에 일자 조정 => 
start_date = '2020-01-01'
end_date = '2025-05-31'
start_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.strptime(end_date, '%Y-%m-%d')
total_days = (end_dt - start_dt).days + 1

# 기타 설정
max_retries = 3
FAIL_LOG = []


# 품목별 반복
for item_name, code in tqdm(ITEM_CODES.items(), desc="전체 품목 진행"):
    LARGE = code[:2]
    MID = code[2:]
    data_list = []

    print(f"\n📦 {item_name} 수집 시작: {start_date} ~ {end_date} (Totl : {total_days})")
    print("📈 진행률: ", end='')

    current_dt = start_dt
    count = 0
    
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')  # API 포맷
        # 휴일이면 제외
        if date_str in holidays:
            continue               
        
        day_success_count = 0
        day_fail_count = 0
        
        for mcode, market_name in df_market.values:
            if str(mcode) in ('210005', '370401'):
                continue
            retry_count = 0
            market_success = False
            page_no = 1

            while retry_count < max_retries:
                try:
                    while True:
                        params = {
                            'serviceKey': API_KEY,
                            'pageNo': page_no,
                            'numOfRows': 300,
                            'cond[trd_clcln_ymd::EQ]': date_str,
                            'cond[whsl_mrkt_cd::EQ]': mcode,
                            'cond[gds_lclsf_cd::EQ]': LARGE,
                            'cond[gds_mclsf_cd::EQ]': MID
                        }

                        response = requests.get(BASE_URL, params=params, verify=False, timeout=10)
                        if response.status_code != 200:
                            fail_reason = f"HTTP {response.status_code}: {response.text[:100]}"
                            retry_count += 1
                            break

                        json_data = response.json()
                        header = json_data.get('response', {}).get('header', {})
                        body = json_data.get('response', {}).get('body', {})
                        items = body.get('items', {}).get('item', [])
                        total_count = int(body.get('totalCount', 0))

                        if isinstance(items, list) and items:
                            data_list.extend(items)
                            market_success = True
                            day_success_count += 1
                        elif isinstance(items, dict):  # 단일 객체일 경우
                            data_list.append(items)
                            market_success = True
                            day_success_count += 1
                        else:
                            break  # 데이터 없음

                        if page_no * 300 >= total_count:
                            break
                        else:
                            page_no += 1

                    break  # 내부 페이지 반복 성공 시 탈출

                except Exception as e:
                    retry_count += 1
                    fail_reason = f"Exception: {str(e)}"
                    time.sleep(0.3)

            if not market_success:
                FAIL_LOG.append({
                    "item": item_name,
                    "market": market_name,
                    "mcode": mcode,
                    "date": date_str,
                    "reason": fail_reason if 'fail_reason' in locals() else 'Unknown'
                })
                day_fail_count += 1

        # 진행률 출력
        count += 1
        
        if count % 30 == 0:
            percent = int((count / total_days) * 100)
            print(f"[누적{count}회({percent}%)-{date_str}]", end='', flush=True)
        else:
            print(f'|O{day_success_count}/X{day_fail_count}|', end='', flush=True)
            
        if count % 300 == 0 and data_list:
            os.makedirs(f"data/{item_name}", exist_ok=True)
            os.makedirs(f"data/{item_name}/backup", exist_ok=True)
            df_temp = pd.DataFrame(data_list)
            filename_temp = f"data/{item_name}/backup/도매시장_MID_{item_name}_{start_date.replace('-', '')}-{date_str.replace('-', '')}({market_name}).csv"
            df_temp.to_csv(filename_temp, encoding='cp949', index=False)
            df_fail_temp = pd.DataFrame(FAIL_LOG)
            df_fail_temp.to_csv(f'data/{item_name}/backup/유통공사_fail_log_중간저장_{count}.csv', index=False, encoding='cp949')
            print(f"💾", end='', flush=True)

        current_dt += timedelta(days=1)
        time.sleep(0.1)

    print(f"\n✅ {item_name} 완료: {count:,}회, {len(data_list):,}건")

    if data_list:
        os.makedirs(f"data/{item_name}", exist_ok=True)
        df = pd.DataFrame(data_list)
        filename = f"data/{item_name}/유통공사_도매시장_{item_name}_{start_date.replace('-', '')}-{end_date.replace('-', '')}.csv"
        df.to_csv(filename, encoding='cp949', index=False)
        print(f"💾 저장됨: {filename}")
        time.sleep(0.3)
    else:
        print(f"⚠️ {item_name}: 수집된 데이터 없음")

    # 실패 로그 저장
    if FAIL_LOG:
        df_fail = pd.DataFrame(FAIL_LOG)
        df_fail.to_csv(f"data/{item_name}/유통공사_fail_log_{item_name}_{start_date.replace('-', '')}-{end_date.replace('-', '')}.csv", index=False, encoding='cp949')
        print(f"\n❗ 실패 요청 {len(FAIL_LOG):,}건 기록됨: data/{item_name}/유통공사_fail_log_{item_name}_{start_date.replace('-', '')}-{end_date.replace('-', '')}.csv")
        time.sleep(0.3)
    else:
        print("\n🎉 모든 수집 성공, 실패 없음!")
print("\n🎉 최종 완료!")

전체 품목 진행:   0%|                                                                            | 0/1 [00:00<?, ?it/s]


📦 감자 수집 시작: 2018-01-01 ~ 2025-05-31 (Totl : 2708)


📈 진행률: |O0/X31||O19/X12||O31/X1||O31/X0||O32/X0||O30/X1||O31/X0||O30/X1||O27/X4||O28/X3||O31/X0||O31/X0||O32/X0||O31/X0||O31/X0||O32/X0||O30/X2||O30/X1||O26/X5||O29/X2||O30/X1||O30/X1||O31/X0||O31/X0||O29/X2||O30/X1||O30/X1||O30/X1||O29/X2|[누적30회(1%)-2018-02-09]|O32/X0||O29/X2||O29/X2||O8/X23||O0/X31||O23/X8||O29/X2||O27/X4||O31/X0||O29/X2||O32/X0||O31/X0||O30/X1||O29/X2||O28/X3||O31/X0||O30/X1||O30/X2||O29/X2||O30/X1||O31/X0||O31/X0||O32/X0||O31/X0||O31/X0||O30/X1||O30/X1||O30/X1||O30/X1|[누적60회(2%)-2018-03-23]|O31/X0||O31/X0||O32/X0||O30/X1||O29/X2||O32/X0||O29/X2||O31/X0||O31/X0||O29/X2||O31/X0||O29/X2||O30/X1||O31/X0||O30/X1||O32/X0||O30/X1||O31/X0||O30/X1||O30/X1||O30/X1||O30/X1||O31/X0||O33/X0||O31/X1||O31/X1||O30/X1||O31/X0||O31/X0|[누적90회(3%)-2018-05-04]|O31/X0||O31/X1||O32/X0||O31/X0||O32/X0||O32/X0||O32/X0||O33/X0||O32/X0||O32/X0||O33/X0||O30/X2||O32/X0||O31/X1||O32/X0||O34/X0||O33/X0||O33/X0||O33/X0||O34/X0||O37/X0||O36/X0||O32/X0||O34/X0||O35/X0||O39/X0||O33/X0||O35/X0||O36/X

In [1]:
data_list

NameError: name 'data_list' is not defined